### Application of Bayesian optimization to weather prediction

In [70]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from numpy import unique
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from tensorflow.keras.layers import SimpleRNN

from sklearn import datasets
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from sklearn.preprocessing import LabelEncoder
from numpy import reshape
from tensorflow.keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets  
from sklearn.ensemble import RandomForestClassifier
from numpy import argmax
 
from sklearn.tree import plot_tree
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score

from keras.utils import to_categorical
from bayes_opt import BayesianOptimization
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedKFold

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from math import floor
from sklearn.metrics import make_scorer, accuracy_score

from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [40]:
path = r'C:\Users\kshifera\Desktop\CF_Course\Specialization\Project\Data Sets'

In [41]:
#Import
station = pd.read_csv(os.path.join(path, 'cleaned_dated.csv'))

In [42]:
station.head()

,Unnamed: 0,DATE,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_cloud_cover,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_pressure,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_humidity,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,0,19600101,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.00,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,0.83,1.0161,0.12,0.08,0.0,10.0,7.0,11.5,7,0.91,1.0010,0.13,0.22,0.0,10.6,9.4,8.3,8,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,1.0304,0.20,0.10,0.0,6.9,1.1,10.4,8,0.98,0.9978,0.04,1.14,0.0,4.9,3.8,5.9,4,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,0.98,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,1,19600102,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,0.84,1.0172,0.25,0.00,0.0,2.9,2.2,4.4,4,0.67,1.017,0.18,0.31,0.0,2.3,1.4,3.1,8,0.90,1.0056,0.14,0.06,0.1,7.7,6.4,8.3,8,0.89,1.0161,0.18,0.66,0.5,8.2,7.4,11.0,7,0.98,1.0051,0.13,0.23,0.0,6.1,3.9,10.6,6,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.94,1.0173,0.56,0.13,3.2,2.1,-1.3,5.5,8,0.92,1.0062,0.17,1.34,0.4,8.6,7.5,9.9,7,0.86,1.0254,0.46,0.0,0.9,9.8,7.4,12.2,6,0.72,1.0292,0.61,0.30,5.1,6.2,4.2,10.2,8,0.62,1.0139,0.04,0.00,0.0,3.4,2.8,4.9,6,0.97,1.0292,0.21,0.61,0.0,-9.5,-10.5,-8.5,5,0.62,1.0114,0.05,0.06,0.0,4.0,3.0,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,2,19600103,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,0.77,1.0179,0.67,0.00,3.5,3.1,-0.5,6.4,4,0.67,1.017

In [43]:
# Drop 'Unnamed: 0' column
station = station.drop(['Unnamed: 0'], axis=1)

In [44]:
#import answers dataframe
picnic = pd.read_csv(os.path.join(path, 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

In [45]:
picnic.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
# Create X and y matrices
X = station.drop(['DATE'], axis=1)
y = picnic.drop(['DATE'], axis=1)

In [47]:
print(X.shape)
print(y.shape)

(22950, 135)
(22950, 15)


In [48]:
# Reshape
X = X.values.reshape(-1,15,9)

In [49]:
print(X.shape)

(22950, 15, 9)


In [50]:
# Reshape
y = y.values.reshape(-1,15)

In [51]:
print(y.shape)

(22950, 15)


In [52]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [53]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(17212, 15, 9)
(5738, 15, 9)
(17212, 15)
(5738, 15)


In [54]:
len(X_train[0])

15

In [55]:
len(X_train[0][0])

9

In [56]:
len(y_train[0])

15

In [58]:
y_train

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int64)

In [59]:
from sklearn.utils.multiclass import type_of_target
type_of_target(y_train)

'multilabel-indicator'

In [60]:
# Use argmax to get rid of on-hot encoding and supply the numerical value
y_train = np.argmax(y_train, axis=1)
print(y_train.shape)
y_train

(17212,)


array([1, 0, 0, ..., 1, 9, 1], dtype=int64)

In [84]:
# Create function
def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]
    optimizer = optimizerL[round(optimizer)] #optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [81]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 
score_acc = make_scorer(accuracy_score)

In [82]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9),
    'optimizer':(0,7),
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000),
    'epochs':(20, 100),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}


In [85]:
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) #25
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/32
15/15 - 3s - 172ms/step - accuracy: 0.6136 - loss: 2.6983
Epoch 2/32
15/15 - 1s - 50ms/step - accuracy: 0.6440 - loss: 2.7004
Epoch 3/32
15/15 - 1s - 48ms/step - accuracy: 0.6440 - loss: 2.6970
Epoch 4/32
15/15 - 1s - 43ms/step - accuracy: 0.6440 - loss: 2.6942
Epoch 5/32
15/15 - 1s - 41ms/step - accuracy: 0.6440 - loss: 2.6916
Epoch 6/32
15/15 - 1s - 41ms/step - accuracy: 0.6440 - loss: 2.6893
Epoch 7/32
15/15 - 1s - 44ms/step - accuracy: 0.6440 - loss: 2.6872
Epoch 8/32
15/15 - 1s - 42ms/step - accuracy: 0.6440 - loss: 2.6852
Epoch 9/32
15/15 - 1s - 43ms/step - accuracy: 0.6440 - loss: 2.6833
Epoch 10/32
15/15 - 1s - 40ms/step - accuracy: 

4/4 - 0s - 58ms/step
Epoch 1/32
15/15 - 2s - 127ms/step - accuracy: 0.5981 - loss: 2.7265
Epoch 2/32
15/15 - 1s - 47ms/step - accuracy: 0.6440 - loss: 2.7004
Epoch 3/32
15/15 - 1s - 44ms/step - accuracy: 0.6440 - loss: 2.6970
Epoch 4/32
15/15 - 1s - 40ms/step - accuracy: 0.6440 - loss: 2.6941
Epoch 5/32
15/15 - 1s - 42ms/step - accuracy: 0.6440 - loss: 2.6916
Epoch 6/32
15/15 - 1s - 41ms/step - accuracy: 0.6440 - loss: 2.6892
Epoch 7/32
15/15 - 1s - 41ms/step - accuracy: 0.6440 - loss: 2.6871
Epoch 8/32
15/15 - 1s - 40ms/step - accuracy: 0.6440 - loss: 2.6851
Epoch 9/32
15/15 - 1s - 40ms/step - accuracy: 0.6440 - loss: 2.6831
Epoch 10/32
15/15 - 1s - 44ms/step - accuracy: 0.6440 - loss: 2.6813
Epoch 11/32
15/15 - 1s - 43ms/step - accuracy: 0.6440 - loss: 2.6795
Epoch 12/32
15/15 - 1s - 41ms/step - accuracy: 0.6440 - loss: 2.6778
Epoch 13/32
15/15 - 1s - 41ms/step - accuracy: 0.6440 - loss: 2.6762
Epoch 14/32
15/15 - 1s - 44ms/step - accuracy: 0.6440 - loss: 2.6746
Epoch 15/32
15/15 - 1

Epoch 10/44
38/38 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 11/44
38/38 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 12/44
38/38 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 13/44
38/38 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 14/44
38/38 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 15/44
38/38 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 16/44
38/38 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 17/44
38/38 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 18/44
38/38 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 19/44
38/38 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 20/44
38/38 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 21/44
38/38 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 22/44
38/38 - 0s - 8ms/step - accuracy: 0.6440 - loss: nan
Epoch 23/44
38/38 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 24/44
38/38 - 0s - 7ms/step - accuracy: 0.6440 - loss: nan
Epoch 25/44
38/38 - 0s - 

Epoch 4/44
38/38 - 0s - 8ms/step - accuracy: 0.6439 - loss: nan
Epoch 5/44
38/38 - 0s - 7ms/step - accuracy: 0.6439 - loss: nan
Epoch 6/44
38/38 - 0s - 7ms/step - accuracy: 0.6439 - loss: nan
Epoch 7/44
38/38 - 0s - 7ms/step - accuracy: 0.6439 - loss: nan
Epoch 8/44
38/38 - 0s - 7ms/step - accuracy: 0.6439 - loss: nan
Epoch 9/44
38/38 - 0s - 7ms/step - accuracy: 0.6439 - loss: nan
Epoch 10/44
38/38 - 0s - 8ms/step - accuracy: 0.6439 - loss: nan
Epoch 11/44
38/38 - 0s - 7ms/step - accuracy: 0.6439 - loss: nan
Epoch 12/44
38/38 - 0s - 11ms/step - accuracy: 0.6439 - loss: nan
Epoch 13/44
38/38 - 0s - 9ms/step - accuracy: 0.6439 - loss: nan
Epoch 14/44
38/38 - 0s - 8ms/step - accuracy: 0.6439 - loss: nan
Epoch 15/44
38/38 - 0s - 9ms/step - accuracy: 0.6439 - loss: nan
Epoch 16/44
38/38 - 0s - 8ms/step - accuracy: 0.6439 - loss: nan
Epoch 17/44
38/38 - 0s - 7ms/step - accuracy: 0.6439 - loss: nan
Epoch 18/44
38/38 - 0s - 7ms/step - accuracy: 0.6439 - loss: nan
Epoch 19/44
38/38 - 0s - 7ms/s

Epoch 10/67
17/17 - 1s - 41ms/step - accuracy: 0.8240 - loss: 0.5079
Epoch 11/67
17/17 - 1s - 46ms/step - accuracy: 0.8281 - loss: 0.4855
Epoch 12/67
17/17 - 1s - 48ms/step - accuracy: 0.8388 - loss: 0.4618
Epoch 13/67
17/17 - 1s - 51ms/step - accuracy: 0.8454 - loss: 0.4403
Epoch 14/67
17/17 - 1s - 49ms/step - accuracy: 0.8500 - loss: 0.4261
Epoch 15/67
17/17 - 1s - 43ms/step - accuracy: 0.8617 - loss: 0.4004
Epoch 16/67
17/17 - 1s - 43ms/step - accuracy: 0.8638 - loss: 0.3890
Epoch 17/67
17/17 - 1s - 42ms/step - accuracy: 0.8705 - loss: 0.3724
Epoch 18/67
17/17 - 1s - 42ms/step - accuracy: 0.8766 - loss: 0.3564
Epoch 19/67
17/17 - 1s - 42ms/step - accuracy: 0.8820 - loss: 0.3424
Epoch 20/67
17/17 - 1s - 43ms/step - accuracy: 0.8807 - loss: 0.3396
Epoch 21/67
17/17 - 1s - 43ms/step - accuracy: 0.8874 - loss: 0.3222
Epoch 22/67
17/17 - 1s - 41ms/step - accuracy: 0.8912 - loss: 0.3204
Epoch 23/67
17/17 - 1s - 42ms/step - accuracy: 0.8921 - loss: 0.3102
Epoch 24/67
17/17 - 1s - 43ms/step

...

Epoch 36/49
18/18 - 1s - 48ms/step - accuracy: 0.8744 - loss: 0.3386
Epoch 37/49
18/18 - 1s - 46ms/step - accuracy: 0.8749 - loss: 0.3393
Epoch 38/49
18/18 - 1s - 47ms/step - accuracy: 0.8683 - loss: 0.3552
Epoch 39/49
18/18 - 1s - 47ms/step - accuracy: 0.8799 - loss: 0.3178
Epoch 40/49
18/18 - 1s - 47ms/step - accuracy: 0.8789 - loss: 0.3258
Epoch 41/49
18/18 - 1s - 48ms/step - accuracy: 0.8830 - loss: 0.3176
Epoch 42/49
18/18 - 1s - 46ms/step - accuracy: 0.8946 - loss: 0.2847
Epoch 43/49
18/18 - 1s - 47ms/step - accuracy: 0.8937 - loss: 0.2888
Epoch 44/49
18/18 - 1s - 48ms/step - accuracy: 0.8842 - loss: 0.3226
Epoch 45/49
18/18 - 1s - 46ms/step - accuracy: 0.8869 - loss: 0.3138
Epoch 46/49
18/18 - 1s - 48ms/step - accuracy: 0.8988 - loss: 0.2753
Epoch 47/49
18/18 - 1s - 47ms/step - accuracy: 0.9075 - loss: 0.2584
Epoch 48/49
18/18 - 1s - 49ms/step - accuracy: 0.9101 - loss: 0.2553
Epoch 49/49
18/18 - 1s - 45ms/step - accuracy: 0.9113 - loss: 0.2529
5/5 - 0s - 49ms/step
Epoch 1/49
18

Epoch 3/61
21/21 - 1s - 25ms/step - accuracy: 0.6938 - loss: 0.8850
Epoch 4/61
21/21 - 1s - 41ms/step - accuracy: 0.7093 - loss: 0.8301
Epoch 5/61
21/21 - 1s - 38ms/step - accuracy: 0.7226 - loss: 0.7886
Epoch 6/61
21/21 - 1s - 28ms/step - accuracy: 0.7355 - loss: 0.7492
Epoch 7/61
21/21 - 1s - 31ms/step - accuracy: 0.7471 - loss: 0.7149
Epoch 8/61
21/21 - 1s - 28ms/step - accuracy: 0.7607 - loss: 0.6836
Epoch 9/61
21/21 - 1s - 25ms/step - accuracy: 0.7698 - loss: 0.6512
Epoch 10/61
21/21 - 1s - 25ms/step - accuracy: 0.7828 - loss: 0.6188
Epoch 11/61
21/21 - 0s - 24ms/step - accuracy: 0.7921 - loss: 0.5873
Epoch 12/61
21/21 - 0s - 22ms/step - accuracy: 0.8043 - loss: 0.5503
Epoch 13/61
21/21 - 1s - 25ms/step - accuracy: 0.8134 - loss: 0.5177
Epoch 14/61
21/21 - 0s - 24ms/step - accuracy: 0.8245 - loss: 0.4919
Epoch 15/61
21/21 - 0s - 24ms/step - accuracy: 0.8332 - loss: 0.4646
Epoch 16/61
21/21 - 0s - 23ms/step - accuracy: 0.8407 - loss: 0.4507
Epoch 17/61
21/21 - 0s - 23ms/step - accu

Epoch 61/61
21/21 - 1s - 41ms/step - accuracy: 0.9541 - loss: 0.1338
6/6 - 1s - 133ms/step
Epoch 1/61
21/21 - 5s - 230ms/step - accuracy: 0.5454 - loss: 1.6956
Epoch 2/61
21/21 - 0s - 23ms/step - accuracy: 0.6440 - loss: 1.0422
Epoch 3/61
21/21 - 1s - 27ms/step - accuracy: 0.6700 - loss: 0.9351
Epoch 4/61
21/21 - 1s - 29ms/step - accuracy: 0.6961 - loss: 0.8601
Epoch 5/61
21/21 - 1s - 32ms/step - accuracy: 0.7228 - loss: 0.7960
Epoch 6/61
21/21 - 1s - 32ms/step - accuracy: 0.7390 - loss: 0.7508
Epoch 7/61
21/21 - 1s - 29ms/step - accuracy: 0.7520 - loss: 0.7161
Epoch 8/61
21/21 - 1s - 27ms/step - accuracy: 0.7612 - loss: 0.6851
Epoch 9/61
21/21 - 1s - 27ms/step - accuracy: 0.7712 - loss: 0.6586
Epoch 10/61
21/21 - 1s - 25ms/step - accuracy: 0.7787 - loss: 0.6299
Epoch 11/61
21/21 - 1s - 28ms/step - accuracy: 0.7901 - loss: 0.6038
Epoch 12/61
21/21 - 1s - 29ms/step - accuracy: 0.7948 - loss: 0.5802
Epoch 13/61
21/21 - 1s - 25ms/step - accuracy: 0.8062 - loss: 0.5500
Epoch 14/61
21/21 - 

Epoch 58/61
21/21 - 0s - 23ms/step - accuracy: 0.9381 - loss: 0.1821
Epoch 59/61
21/21 - 0s - 24ms/step - accuracy: 0.9386 - loss: 0.1768
Epoch 60/61
21/21 - 0s - 20ms/step - accuracy: 0.9434 - loss: 0.1660
Epoch 61/61
21/21 - 0s - 23ms/step - accuracy: 0.9419 - loss: 0.1674
6/6 - 0s - 43ms/step
Epoch 1/61
21/21 - 2s - 85ms/step - accuracy: 0.4582 - loss: 1.9328
Epoch 2/61
21/21 - 0s - 21ms/step - accuracy: 0.6590 - loss: 1.0545
Epoch 3/61
21/21 - 0s - 17ms/step - accuracy: 0.6886 - loss: 0.9356
Epoch 4/61
21/21 - 0s - 19ms/step - accuracy: 0.7059 - loss: 0.8745
Epoch 5/61
21/21 - 0s - 20ms/step - accuracy: 0.7200 - loss: 0.8278
Epoch 6/61
21/21 - 0s - 18ms/step - accuracy: 0.7311 - loss: 0.7852
Epoch 7/61
21/21 - 0s - 20ms/step - accuracy: 0.7436 - loss: 0.7437
Epoch 8/61
21/21 - 0s - 21ms/step - accuracy: 0.7548 - loss: 0.7070
Epoch 9/61
21/21 - 0s - 17ms/step - accuracy: 0.7660 - loss: 0.6737
Epoch 10/61
21/21 - 0s - 16ms/step - accuracy: 0.7738 - loss: 0.6381
Epoch 11/61
21/21 - 0s

Traceback (most recent call last):
  File "C:\Users\kshifera\AppData\Local\anaconda3\Lib\site-packages\bayes_opt\bayesian_optimization.py", line 306, in maximize
    x_probe = next(self._queue)
              ^^^^^^^^^^^^^^^^^
  File "C:\Users\kshifera\AppData\Local\anaconda3\Lib\site-packages\bayes_opt\bayesian_optimization.py", line 27, in __next__
    raise StopIteration("Queue is empty, no more objects to retrieve.")
StopIteration: Queue is empty, no more objects to retrieve.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\kshifera\AppData\Local\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\kshifera\AppData\Local\Temp\ipykernel_22352\1681450158.py", line 3, in <module>
    nn_opt.maximize(init_points=15, n_iter=4) #25
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\kshifera\AppData\Loca

In [86]:
best_params = nn_opt.max['params']
best_score = nn_opt.max['target']

print(f"Best Parameters: {best_params}")
print(f"Highest Accuracy: {best_score}")

Best Parameters: {'activation': 2.79884089544096, 'batch_size': 460.14665762139765, 'dropout': 0.7296061783380641, 'dropout_rate': 0.19126724140656393, 'epochs': 90.97701940610612, 'kernel': 1.9444298503238986, 'layers1': 1.2391884918766034, 'layers2': 2.42648957444599, 'learning_rate': 0.7631771981307285, 'neurons': 60.51494778125466, 'normalization': 0.770967179954561, 'optimizer': 3.456569174550735}
Highest Accuracy: nan


In [87]:
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', 'LeakyReLU', 'relu']
optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'SGD']

# Best parameters from optimization
activation_index = round(best_params['activation'])
optimizer_index = round(best_params['optimizer'])

best_params['activation'] = activationL[activation_index]
best_params['optimizer'] = optimizerL[optimizer_index]
best_params['neurons'] = round(best_params['neurons'])
best_params['kernel'] = round(best_params['kernel'])
best_params['batch_size'] = round(best_params['batch_size'])
best_params['epochs'] = round(best_params['epochs'])
best_params['layers1'] = round(best_params['layers1'])
best_params['layers2'] = round(best_params['layers2'])

print(f"Best Parameters (interpreted): {best_params}")

Best Parameters (interpreted): {'activation': 'softsign', 'batch_size': 460, 'dropout': 0.7296061783380641, 'dropout_rate': 0.19126724140656393, 'epochs': 91, 'kernel': 2, 'layers1': 1, 'layers2': 2, 'learning_rate': 0.7631771981307285, 'neurons': 61, 'normalization': 0.770967179954561, 'optimizer': 'Adadelta'}


In [88]:
# Best parameters from optimization
best_params = {
    'neurons': 61,
    'kernel': 2,
    'activation': 'softsign',
    'optimizer': 'Adadelta',
    'learning_rate': 0.76317719813,
    'batch_size': 460,
    'epochs': 91,
    'layers1': 1,
    'layers2': 2,
    'normalization': 0.770967179954561,
    'dropout': 0.7296061783380641,
    'dropout_rate':  0.19126724140656393
}



# Initialize optimizer with learning rate
optimizers = {
    'Adam': Adam(learning_rate=best_params['learning_rate']),
    'SGD': SGD(learning_rate=best_params['learning_rate']),
    'RMSprop': RMSprop(learning_rate=best_params['learning_rate']),
    'Adadelta': Adadelta(learning_rate=best_params['learning_rate']),
    'Adagrad': Adagrad(learning_rate=best_params['learning_rate']),
    'Adamax': Adamax(learning_rate=best_params['learning_rate']),
    'Nadam': Nadam(learning_rate=best_params['learning_rate']),
    'Ftrl': Ftrl(learning_rate=best_params['learning_rate'])
}

optimizer = optimizers[best_params['optimizer']]

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
#n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(best_params['neurons'], kernel_size=best_params['kernel'], activation=best_params['activation'], input_shape=(15,9)))

if best_params['normalization'] > 0.5:
    model.add(BatchNormalization())

for _ in range(best_params['layers1']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))

if best_params['dropout'] > 0.5:
    model.add(Dropout(best_params['dropout_rate'], seed=123))

for _ in range(best_params['layers2']):
    model.add(Dense(best_params['neurons'], activation=best_params['activation']))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) # sigmoid, tanh, softmax

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model with the optimized parameters
model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=2)

Epoch 1/91
38/38 - 2s - 51ms/step - accuracy: 0.6088 - loss: 1.3571
Epoch 2/91
38/38 - 1s - 20ms/step - accuracy: 0.6976 - loss: 0.8980
Epoch 3/91
38/38 - 1s - 19ms/step - accuracy: 0.7287 - loss: 0.8045
Epoch 4/91
38/38 - 1s - 20ms/step - accuracy: 0.7401 - loss: 0.7582
Epoch 5/91
38/38 - 1s - 18ms/step - accuracy: 0.7560 - loss: 0.7162
Epoch 6/91
38/38 - 1s - 20ms/step - accuracy: 0.7699 - loss: 0.6746
Epoch 7/91
38/38 - 1s - 17ms/step - accuracy: 0.7821 - loss: 0.6447
Epoch 8/91
38/38 - 1s - 18ms/step - accuracy: 0.7878 - loss: 0.6173
Epoch 9/91
38/38 - 1s - 17ms/step - accuracy: 0.7984 - loss: 0.5870
Epoch 10/91
38/38 - 1s - 18ms/step - accuracy: 0.8065 - loss: 0.5602
Epoch 11/91
38/38 - 1s - 18ms/step - accuracy: 0.8140 - loss: 0.5362
Epoch 12/91
38/38 - 1s - 17ms/step - accuracy: 0.8256 - loss: 0.5030
Epoch 13/91
38/38 - 1s - 18ms/step - accuracy: 0.8305 - loss: 0.4866
Epoch 14/91
38/38 - 1s - 21ms/step - accuracy: 0.8395 - loss: 0.4668
Epoch 15/91
38/38 - 1s - 19ms/step - accura

In [89]:
model.summary()

Model: "sequential_75"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_75 (Conv1D)                   │ (None, 14, 61)              │           1,159 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_30               │ (None, 14, 61)              │             244 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_380 (Dense)                    │ (None, 14, 61)              │           3,782 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_45 (Dropout)                 │ (None, 14, 61)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_381 (Dense)                    │ (None, 14, 61)              │           3,782 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_382 (Dense)                    │ (None, 14, 61)              │           3,782 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_75 (MaxPooling1D)      │ (None, 7, 61)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_75 (Flatten)                 │ (None, 427)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_383 (Dense)                    │ (None, 15)                  │           6,420 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 57,265 (223.70 KB)

 Trainable params: 19,047 (74.40 KB)

 Non-trainable params: 122 (488.00 B)

 Optimizer params: 38,096 (148.82 KB)

In [90]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [91]:
y_train_h = to_categorical(y_train, num_classes=15)

In [92]:
# Check shapes
print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_h.shape}')

X_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [93]:
model.fit(X_train, y_train_h, batch_size=460, epochs=91, verbose=2)

Epoch 1/91
38/38 - 2s - 56ms/step - accuracy: 0.9554 - loss: 0.1315
Epoch 2/91
38/38 - 1s - 23ms/step - accuracy: 0.9679 - loss: 0.0961
Epoch 3/91
38/38 - 1s - 22ms/step - accuracy: 0.9644 - loss: 0.1038
Epoch 4/91
38/38 - 1s - 21ms/step - accuracy: 0.9669 - loss: 0.0976
Epoch 5/91
38/38 - 1s - 20ms/step - accuracy: 0.9661 - loss: 0.1041
Epoch 6/91
38/38 - 1s - 20ms/step - accuracy: 0.9650 - loss: 0.1027
Epoch 7/91
38/38 - 1s - 19ms/step - accuracy: 0.9681 - loss: 0.0934
Epoch 8/91
38/38 - 1s - 20ms/step - accuracy: 0.9697 - loss: 0.0920
Epoch 9/91
38/38 - 1s - 19ms/step - accuracy: 0.9708 - loss: 0.0822
Epoch 10/91
38/38 - 1s - 18ms/step - accuracy: 0.9715 - loss: 0.0864
Epoch 11/91
38/38 - 1s - 19ms/step - accuracy: 0.9689 - loss: 0.0882
Epoch 12/91
38/38 - 1s - 19ms/step - accuracy: 0.9716 - loss: 0.0852
Epoch 13/91
38/38 - 1s - 20ms/step - accuracy: 0.9700 - loss: 0.0837
Epoch 14/91
38/38 - 1s - 20ms/step - accuracy: 0.9694 - loss: 0.0868
Epoch 15/91
38/38 - 1s - 19ms/step - accura

In [94]:
stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [95]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [96]:
y_pred = model.predict(X_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [97]:
cm = confusion_matrix(y_test, y_pred, stations)
print(cm)

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        3647        17         4       0           3         1       0   
BELGRADE       30      1059         1       0           1         0       0   
BUDAPEST        8         3       201       1           0         1       0   
DEBILT          2         2         3      75           0         0       0   
DUSSELDORF      1         0         0       4          18         6       0   
HEATHROW        2         0         0       2           3        74       0   
KASSEL          0         0         0       0           0         0      10   
LJUBLJANA       5         2         0       0           0         0       0   
MAASTRICHT      2         0         0       0           0         1       0   
MADRID         17         2         2       1           2         3       0   
MUNCHENB        4         1         0       0       